<a href="https://www.ired.info/"> <img src="file/ired_logo.png" alt="ired" style="width: 150px;"/> </a>

# Object Classification- Test Model

### Initiate PyTorch

In [ ]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

### Select model for testing

In [ ]:
model.load_state_dict(torch.load('ired_model.pth'))

### Display detection result

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

##

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
press_slider = widgets.FloatSlider(description='press', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, press_slider]))

##

import torch.nn.functional as F
import time
old_time=time.time()

def update(change):
    global press_slider, old_time, prob
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    y = F.softmax(y, dim=1)
    
    prob_press = float(y.flatten()[0])
    press_slider.value = prob_press
    
    time.sleep(0.001)
        
update({'new': camera.value})

camera.observe(update, names='value')

### Fill in your own thershold value (default: 0.5)

### Threshold value : 0.xx

### Stop camera

In [ ]:
camera.stop()

--End--